In [4]:
from skdata.mnist.views import OfficialVectorClassification
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import pandas as pd

In [5]:
data = OfficialVectorClassification()

In [6]:
trIdx = data.sel_idxs[:]

In [7]:
df_labels = pd.get_dummies(data.all_labels)

In [ ]:
writer = tf.python_io.TFRecordWriter("mnist.tfrecords")

In [12]:
# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
num_steps = 28 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# Parameters
learning_rate = 0.001

In [ ]:
for example_idx in tqdm(trIdx):
    features = data.all_vectors[example_idx]
    label = df_labels.iloc[example_idx]
    # construct the Example proto boject
    example = tf.train.Example(
        # Example contains a Features proto object
        features=tf.train.Features(
          # Features contains a map of string to Feature proto objects
          feature={
            # A Feature contains one of either a int64_list,
            # float_list, or bytes_list
            'label': tf.train.Feature(
                float_list=tf.train.FloatList(value=features.astype("float"))),
            'image': tf.train.Feature(
                float_list=tf.train.FloatList(value=features.astype("float"))),
    }))
    
    # use the proto object to serialize the example to a string
    serialized = example.SerializeToString()
    # write the serialized object to disk
    writer.write(serialized)

In [8]:
filename_queue = tf.train.string_input_producer(["mnist.tfrecords"], num_epochs=None)

In [9]:
reader = tf.TFRecordReader()

In [10]:
_, serialized_example = reader.read(filename_queue)

In [13]:
features = tf.parse_single_example(
        serialized_example,
        features={
            # We know the length of both fields. If not the
            # tf.VarLenFeature could be used
            'label': tf.FixedLenFeature([num_classes], tf.float32),
            'image': tf.FixedLenFeature([num_input*num_steps], tf.float32)
        })

In [14]:
label = features['label']
image = features['image']

In [26]:
tf.reshape(images_batch, [num_hidden, num_input, num_steps])

Tensor("ParseSingleExample/Squeeze_image:0", shape=(784,), dtype=float32)


In [16]:
# groups examples into batches randomly
images_batch, labels_batch = tf.train.shuffle_batch(
    [image, label], batch_size=num_hidden,
    capacity=2000,
    min_after_dequeue=1000)

In [17]:
print images_batch.get_shape()
print labels_batch.get_shape()

(128, 784)
(128, 10)


In [18]:
images_batch = tf.reshape(images_batch, [num_hidden, num_input, num_steps])

In [19]:
labels_batch = tf.reshape(labels_batch, [num_hidden, num_classes])

In [20]:
print images_batch.get_shape()
print labels_batch.get_shape()

(128, 28, 28)
(128, 10)


In [21]:
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)
val, state = tf.nn.dynamic_rnn(cell, images_batch, dtype=tf.float32)

In [22]:
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

weight = tf.Variable(tf.truncated_normal([num_hidden, num_classes]))
bias = tf.Variable(tf.truncated_normal([num_classes]))

# Functions definition 
pred = tf.matmul(last, weight) + bias

# Define loss and optimizer
cost = tf.sqrt(tf.reduce_mean(tf.square(labels_batch - pred)), name='RMSE')
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(labels_batch, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [23]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-4, started daemon 123145330311168)>,
 <Thread(Thread-5, started daemon 123145334517760)>]

ERROR:tensorflow:Exception in QueueRunner: Name: <unknown>, Key: label, Index: 0.  Number of float values != expected.  values size: 784 but output shape: [10]
	 [[Node: ParseSingleExample/ParseExample/ParseExample = ParseExample[Ndense=2, Nsparse=0, Tdense=[DT_FLOAT, DT_FLOAT], dense_shapes=[[784], [10]], sparse_types=[], _device="/job:localhost/replica:0/task:0/cpu:0"](ParseSingleExample/ExpandDims, ParseSingleExample/ParseExample/ParseExample/names, ParseSingleExample/ParseExample/ParseExample/dense_keys_0, ParseSingleExample/ParseExample/ParseExample/dense_keys_1, ParseSingleExample/ParseExample/Const, ParseSingleExample/ParseExample/Const_1)]]
Caused by op u'ParseSingleExample/ParseExample/ParseExample', defined at:
  File "/usr/local/Cellar/python/2.7.12/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/Cellar/python/2.7.12/Frameworks/Python.framework/Versions/2.7/lib/pyth

In [ ]:
for i in range(10):
  # pass it in through the feed_dict
  _, loss_val = sess.run([optimizer, cost])
  print loss_val

In [ ]:
filename = "mnist.tfrecords"
for serialized_example in tf.python_io.tf_record_iterator(filename):
    example = tf.train.Example()
    example.ParseFromString(serialized_example)

    # traverse the Example format to get data
    image = example.features.feature['image'].float_list.value
    label = example.features.feature['label'].float_list.value[0]
    # do something
    # print image, label

In [ ]:
label

In [ ]:
image_input = np.array(image)

In [ ]:
image_input.shape

In [ ]:
zeros = np.zeros((127, 784))
test = np.concatenate((image_input, zeros), axis=0)

In [ ]:
test.shape

In [ ]:
image_input = image_input.reshape(1, 784)
image_input.shape

In [ ]:
feed_dict = {images_batch: test.reshape(128, 784, 1)}
classification = sess.run(prediction, feed_dict)
print classification